In [2]:
import pandas as pd
import numpy as np

season_data = pd.read_csv("pbp_2017.csv", low_memory=False)



In [3]:
#Relevant Columns

cols = ['Date',
'GameID',
'Drive',
'qtr',
'down',
'TimeSecs',
'PlayTimeDiff',
'SideofField',
'yrdline100',
'ydstogo',
'ydsnet',
'GoalToGo',
'FirstDown',
'posteam',
'DefensiveTeam',
'PlayAttempted',
'Yards.Gained',
'sp',
'Touchdown',
'ExPointResult',
'TwoPointConv',
'DefTwoPoint',
'Safety',
'PlayType',
'Passer',
'Passer_ID',
'PassAttempt',
'PassOutcome',
'PassLength',
'AirYards',
'YardsAfterCatch',
'QBHit',
'PassLocation',
'InterceptionThrown',
'Rusher',
'Rusher_ID',
'RushAttempt',
'RunLocation',
'RunGap',
'Receiver',
'Receiver_ID',
'Reception',
'Fumble',
'RecFumbTeam',
'Sack',
'PosTeamScore',
'DefTeamScore',
'ScoreDiff',
'AbsScoreDiff',
'HomeTeam',
'AwayTeam',
'Season']

season_data = season_data[cols]

In [24]:
#Just Defensive Score Points

def score_against(team):
    defense_df = season_data[season_data['DefensiveTeam']==team]
    game_list = defense_df['GameID'].unique().tolist()
    
    score_points = 0
    
    for game in game_list:
        game_df = defense_df[defense_df['GameID']==game]
        points_given_up = game_df['PosTeamScore'].max()
        
        if points_given_up == 0:
            score_points += 10
        elif points_given_up >= 1 and points_given_up <= 6:
            score_points += 7
        elif points_given_up >= 7 and points_given_up <= 13:
            score_points += 4
        elif points_given_up >= 14 and points_given_up <= 20:
            score_points += 1
        elif points_given_up >= 21 and points_given_up <= 27:
            score_points += 0
        elif points_given_up >= 28 and points_given_up <= 34:
            score_points -= 1
        else:
            score_points -= 4
 
    return(score_points) 



In [25]:
#TOTAL Season Defense Points
#TEAM ABBREVIATIONS
defense_list = ['KC','NE','NYJ','BUF','PHI','WAS','TEN','OAK','HOU','JAX','DET','ARI','PIT','CLE','CHI','ATL','BAL','CIN','IND','LA','SF','CAR','GB','SEA','NYG','DAL','MIN','NO','LAC','DEN','TB','MIA']

def team_defense(team):
    defense_df = season_data[season_data['DefensiveTeam']==team]
    touchdown_df = defense_df[defense_df['Touchdown'] == 1]
    
    #SACKS
    total_sacks = defense_df['Sack'].sum()
    
    #INTERCEPTIONS
    total_interceptions = defense_df['InterceptionThrown'].sum()
    total_interception_points = 2 * total_interceptions
    
    #TDs OFF INTERCEPTIONS
    int_return_td = touchdown_df['InterceptionThrown'].sum()
    int_return_td_points = 6 * int_return_td

    #FUMBLE RECOVERIES
    total_recoveries = 0
    fumble_series = defense_df['RecFumbTeam'].dropna()

    for recovery in fumble_series:
        if recovery == team:
            total_recoveries += 1
    fumble_recovery_points = 2 * total_recoveries 

    #SAFETY
    total_safeties = defense_df['Safety'].sum()
    
    safety_points = 2 * total_safeties
    
    #Defensive Score Points
    total_defensive_score_points = score_against(team)
    
    #TOTAL DEFENSIVE POINTS
    total_dk_defense_points = total_sacks + total_interception_points + int_return_td_points + fumble_recovery_points + safety_points + total_defensive_score_points
    return(total_dk_defense_points)

DST_ranks = {}
for team in defense_list:
    points = team_defense(team)
    DST_ranks[team] = points

#Produces dataframe of just Defenses and their overall season points
DST_ranks_df = pd.DataFrame.from_dict(DST_ranks, orient='index', columns=['Total Points'])
pd.to_numeric(DST_ranks_df['Total Points'])

#Prettying Up The Data Frame Holy Shit This Was Annoying
DST_ranks_df = DST_ranks_df['Total Points'].sort_values(ascending=False)
DST =  pd.DataFrame(data=DST_ranks_df)
DST['Team'] = DST.index
DST.reset_index(inplace=True, level=None)
DST = DST.drop(['index'], axis=1)
DST = DST[['Team', 'Total Points']]


In [26]:
DST

,Team,Total Points
0,BAL,167
1,JAX,156
2,LA,152
3,LAC,149
4,PHI,138
5,NO,135
6,SEA,133
7,DET,131
8,PIT,128
9,CHI,123


In [32]:
#Season Broken into weeks
week_one = ['2017-09-07','2017-09-10','2017-09-11']
week_two = ['2017-09-14', '2017-09-17','2017-09-18' ]
week_three = ['2017-09-21', '2017-09-24', '2017-09-25']
week_four = ['2017-09-28', '2017-10-01', '2017-10-02']
week_five = ['2017-10-05', '2017-10-08', '2017-10-09']
week_six = ['2017-10-12', '2017-10-15', '2017-10-16']
week_seven = ['2017-10-19', '2017-10-22', '2017-10-23']
week_eight = ['2017-10-26', '2017-10-29', '2017-10-30']
week_nine = ['2017-11-02', '2017-11-05', '2017-11-06' ]
week_ten = ['2017-11-09', '2017-11-12', '2017-11-13']
week_eleven = ['2017-11-16', '2017-11-19', '2017-11-20']
week_twelve = ['2017-11-23', '2017-11-26', '2017-11-27']
week_thirteen = ['2017-11-30', '2017-12-03', '2017-12-04']
week_fourteen = ['2017-12-07', '2017-12-10', '2017-12-11']
week_fifteen = ['2017-12-14', '2017-12-16', '2017-12-17']
week_sixteen = ['2017-12-18', '2017-12-23', '2017-12-24', '2017-12-25']
week_seventeen = ['2017-12-31']

all_the_weeks_past_one = [week_two, week_three, week_four, week_five, week_six, week_seven, week_eight, week_nine, week_ten, week_eleven, week_twelve, week_thirteen, week_fourteen, week_fifteen, week_sixteen, week_seventeen]

def weekly_defense_points(team, week):

    current_week_df = pd.DataFrame()
    for game_day in week:
        game_day_df = season_data[season_data['Date']==game_day]
        current_week_df = pd.concat([current_week_df, game_day_df], ignore_index=True)
    
    #Defense Time Again $
    defense_df = current_week_df[current_week_df['DefensiveTeam']==team]
    defense_pass_play = defense_df[defense_df['PassOutcome']=='Complete']
    #Rushing Stuff $
    pd.to_numeric(current_week_df['RushAttempt'])
    defense_rush_play = defense_df[defense_df['RushAttempt']==1]
    
    opponent = str(defense_df['posteam'].unique())
    opponent = opponent.replace("'","")
    opponent = opponent.replace("[", "")
    opponent = opponent.replace("]","")
    
    #Home and Away Teams $
    away_team = str(defense_df['AwayTeam'].unique())
    away_team = away_team.replace("'","")
    away_team = away_team.replace("[", "")
    away_team = away_team.replace("]","")
    home_team = str(defense_df['HomeTeam'].unique())
    home_team = home_team.replace("'","")
    home_team = home_team.replace("[", "")
    home_team = home_team.replace("]","")
    
    #Total Passing Yards $
    pd.to_numeric(defense_pass_play['AirYards'])
    pd.to_numeric(defense_pass_play['YardsAfterCatch'])
    
    total_pass_yards = defense_pass_play['AirYards'].sum() + defense_pass_play['YardsAfterCatch'].sum()
    
    #Interceptions $
    pd.to_numeric(defense_df['InterceptionThrown'])
    total_interceptions = defense_df['InterceptionThrown'].sum()
    interception_points = total_interceptions * 2
    
    interception_td_df = defense_df[(defense_df['InterceptionThrown']==1) & (defense_df['Touchdown']==1)]
    interception_td = len(interception_td_df)
    interception_td_points = interception_td * 6
    
    #Passing Touchdowns $
    pd.to_numeric(defense_pass_play['Touchdown'])
    total_pass_td = defense_pass_play['Touchdown'].sum()
    
    #Rushing Touchdowns $
    pd.to_numeric(defense_rush_play['Touchdown'])
    total_rushing_touchdowns = defense_rush_play['Touchdown'].sum()
    
    #Rushing Yards $
    pd.to_numeric(defense_rush_play['Yards.Gained'])
    total_rushing_yds = defense_rush_play['Yards.Gained'].sum()
    
    #Fumbles $
    pd.to_numeric(defense_df['Fumble'])
    fumble_df = defense_df[(defense_df['Fumble']==1) & (defense_df['RecFumbTeam']==team)]
    
    total_fumble_rec = len(fumble_df)
    fumble_points = total_fumble_rec * 2
    
    fumble_td_df = fumble_df[fumble_df['Touchdown']==1]
    fumble_td = len(fumble_td_df)
    fumble_td_points = fumble_td * 6
    
    #Receiving Touchdowns
    pd.to_numeric(defense_pass_play['Touchdown'])
    total_receiving_touchdowns = defense_pass_play['Touchdown'].sum()
    
    #Receiving Yards
    total_receiving_yards = defense_pass_play['AirYards'].sum() + defense_pass_play['YardsAfterCatch'].sum()
    
    #PPR
    receptions = len(defense_pass_play)
    
    #Sacks
    total_sacks = defense_df['Sack'].sum()
    sack_points = total_sacks
    
    #Safety
    total_safety = defense_df['Safety'].sum()
    safety_points = total_safety * 2
    
    defense_tds = fumble_td + interception_td
    
    #Motherfucking Conditionals 
    game_df = defense_df.dropna(subset=['PosTeamScore'])
    points_given_up = game_df['PosTeamScore'].max()
    
    score_points = 0
    
    if points_given_up == 0:
        score_points += 10
    elif points_given_up >= 1 and points_given_up <= 6:
        score_points += 7
    elif points_given_up >= 7 and points_given_up <= 13:
        score_points += 4
    elif points_given_up >= 14 and points_given_up <= 20:
        score_points += 1
    elif points_given_up >= 21 and points_given_up <= 27:
        score_points += 0
    elif points_given_up >= 28 and points_given_up <= 34:
        score_points -= 1
    else:
        score_points -= 4    
    
    #Sum that shit up
    total_week_points = fumble_td_points + sack_points + interception_points + fumble_points + safety_points + interception_td_points + score_points
    
    weekly_defense_stats = [total_week_points, total_sacks, total_interceptions, total_fumble_rec, total_rushing_yds, total_rushing_touchdowns, total_receiving_yards, total_receiving_touchdowns, opponent, home_team, away_team]
    
    return(weekly_defense_stats)


In [33]:
#This still needs to be done!!!

def dst_season_averages(team):
    defense_df = season_data[season_data['DefensiveTeam']==team]
    defense_pass_play = defense_df[defense_df['PassOutcome']=='Complete']
    pd.to_numeric(defense_df['RushAttempt'])
    defense_rush_play = defense_df[defense_df['RushAttempt']==1]
    
    #Total Passing Yards
    pd.to_numeric(defense_pass_play['AirYards'])
    pd.to_numeric(defense_pass_play['YardsAfterCatch'])
    
    total_pass_yards = defense_pass_play['AirYards'].sum() + defense_pass_play['YardsAfterCatch'].sum()
    average_pass_yards = total_pass_yards / float(16)
    
    #Deep Ball Passes
    deep_ball_pass_df = defense_pass_play[defense_pass_play['PassLength']=='Deep']
    deep_completions = len(deep_ball_pass_df)
    avg_deep_completions = deep_completions / float(16)
    
    #Interceptions
    pd.to_numeric(defense_df['InterceptionThrown'])
    total_interceptions = defense_df['InterceptionThrown'].sum()
    average_interceptions = total_interceptions / float(16)
    
    #Passing Touchdowns
    pd.to_numeric(defense_pass_play['Touchdown'])
    total_pass_td = defense_pass_play['Touchdown'].sum()
    average_pass_tds = total_pass_td / float(16)
    
    #Rushing Touchdowns
    pd.to_numeric(defense_rush_play['Touchdown'])
    total_rushing_touchdowns = defense_rush_play['Touchdown'].sum()
    average_rush_tds = total_rushing_touchdowns / float(16)
    
    #Rushing Yards
    pd.to_numeric(defense_rush_play['Yards.Gained'])
    total_rushing_yards = defense_rush_play['Yards.Gained'].sum()
    average_rush_yards = total_rushing_yards / float(16)
    
    #Red Zone Opportunities
    pd.to_numeric(defense_pass_play['yrdline100'])
    pd.to_numeric(defense_rush_play['yrdline100'])
    red_zone_pass_df = defense_pass_play[defense_pass_play['yrdline100'] >= 80]
    red_zone_rush_df = defense_rush_play[defense_rush_play['yrdline100'] >= 80]
    red_zone_pass_opps = len(red_zone_pass_df)
    red_zone_rush_opps = len(red_zone_rush_df) 
    red_zone_opps = red_zone_pass_opps + red_zone_rush_opps
    #Avg Red Zone Opps
    avg_red_zone_opps = red_zone_opps / float(16)
    
    dst_averages = [average_pass_yards, average_pass_tds, avg_deep_completions, average_interceptions, average_rush_yards, average_rush_tds, avg_red_zone_opps]

    return(dst_averages)

defense_list = DST['Team'].tolist()

def average_columns(val):
    average_calc = []
    for dst in defense_list:
        average_calc.append(dst_season_averages(dst)[val])
        
    return(average_calc)

DST['Average Receiving Yards Given'] = average_columns(0)
DST['Average Receiving TDs Given'] = average_columns(1)
DST['Average Deep Ball Completions'] = average_columns(2)
DST['Average Interceptions'] = average_columns(3)
DST['Average Rush Yards Given'] = average_columns(4)
DST['Average Rush TDs'] = average_columns(5)
DST['Average Red Zone Opps'] = average_columns(6)




    

In [34]:
#defense weekly data

def column_creator(val):
    current_week_pts = []
    for dst in defense_list:
        current_week_pts.append(weekly_defense_points(dst,week_one)[val])

    return(current_week_pts)

DST['Week 1 pts'] = column_creator(0)
DST['Week 1 Sacks'] = column_creator(1)
DST['Week 1 Interceptions'] = column_creator(2)
DST['Week 1 Fumble Recoveries'] = column_creator(3)
DST['Week 1 Rush Yds Given'] = column_creator(4)
DST['Week 1 Rush TDs Given'] = column_creator(5)
DST['Week 1 Receiving Yds Given'] = column_creator(6)
DST['Week 1 Receiving TDs Given'] = column_creator(7)
DST['Week 1 Opponent'] = column_creator(8)
DST['Week 1 Home Team'] = column_creator(9)
DST['Week 1 Away Team'] = column_creator(10)

for i in range(2,17):
    week = all_the_weeks_past_one[i-2]
    def column_creator_v2(val):
        current_week_pts = []
        for dst in defense_list:
            current_week_pts.append(weekly_defense_points(dst, week)[val])
        
        return(current_week_pts)

    DST['Week '+ str(i) + ' pts'] = column_creator_v2(0)
    DST['Week '+ str(i) + ' Sacks'] = column_creator_v2(1)
    DST['Week '+ str(i) + ' Interceptions'] = column_creator_v2(2)
    DST['Week '+ str(i) + ' Fumble Recoveries'] = column_creator_v2(3)
    DST['Week '+ str(i) + ' Rush Yds Given'] = column_creator_v2(4)
    DST['Week '+ str(i) + ' Rush TDs Given'] = column_creator_v2(5)
    DST['Week '+ str(i) + ' Receiving Yds Given'] = column_creator_v2(6)
    DST['Week '+ str(i) + ' Receiving TDs Given'] = column_creator_v2(7)
    DST['Week '+ str(i) + ' Opponent'] = column_creator_v2(8)
    DST['Week '+ str(i) + ' Home Team'] = column_creator_v2(9)
    DST['Week '+ str(i) + ' Away Team'] = column_creator_v2(10)
        



In [35]:
DST.to_csv('defense_2017_v2.csv', sep=',')